In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
p = Path('.')
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list


In [22]:
json_string='''@article{anderson2006empirical,
  title={Empirical evidence on capital investment, growth options, and security returns},
  author={Anderson, Christopher W and Garcia-Feijoo, Luis},
  journal={The journal of finance},
  volume={61},
  number={1},
  pages={171--194},
  year={2006},
  publisher={Wiley Online Library}
}'''

url= f'https://academic.oup.com/rfs/article-abstract/33/5/2019/5236964?redirectedFrom=PDF' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (HTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')
#dsoup = copy.deepcopy(soup)

# driver = webdriver.Chrome('/usr/local/bin/chromedriver')
# driver.get(url)
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(driver.page_source, "html.parser")
# soup=BeautifulSoup(citation_dict['cited_by'].text,'html.parser')

citation_dict={}
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=cited_by_gscholar(soup)
df = pd.DataFrame.from_dict([citation_dict])
df.to_csv('academic_citation_network.csv', mode='a')

# driver.close()

!clear
!git status
!export citation_key=($citation_key)
!git add .
!git commit -m "$citation_key"
!git push

# docker run --rm --init --ulimit core=0 -p 8070:8070 lfoppiano/grobid:0.8.0


article--2006--Anderson-Christopher-W-and-Garcia-Feijoo-Luis--Empirical-evidence-on-capital-investment-growth-options-and-security-returns--The-journal-of-finance.pdf
CITATION KEY:  anderson2006empirical
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
import os
import xml.etree.ElementTree as ET
tree = ET.parse('/Users/krishnaneupane/Downloads/testGorbid/ssrn-3604626.grobid.tei.xml')
root = tree.getroot()
for item in root.keys():
    print(item)

In [ ]:
import xmltojson
from pprint import pprint
import json 

import xmltojson
with open('/Users/krishnaneupane/Downloads/testGorbid/ssrn-3604626.grobid.tei.xml', 'r') as f:
    my_xml = f.read()
dd= json.loads(xmltojson.parse(my_xml))

for item in dd['TEI']['text']['back']['div']:
    if 'listBibl' in list(item.keys()):

        for sitem in item['listBibl']['biblStruct']:
            journal = sitem.get('monogr').get('title').get('#text')
            try: 
                title= sitem['analytic']['title']['#text']
                for stitle in sitem['analytic']['author']:
                    fname= stitle.get('persName').get('forename')
                    if  isinstance(fname, list):
                        flname=[i['#text'] for i in fname]
                        lname= stitle['persName'].get('surname')
                        flname= flname + [lname]
                        flname= ' '.join(flname)
                        print(title,flname, journal)
                    else:
                        fname= stitle.get('persName').get('forename').get('#text')
                        lname= stitle['persName'].get('surname')
                        flname= fname +' '+ lname
                        print(title, flname, journal)
            except Exception:
                pass
            


In [ ]:
!pip install xmltojson